In [1]:
!git clone -b mixtral https://github.com/cg123/mergekit.git
!cd mergekit && pip install -qqq -e . --progress-bar off
!pip install -qqq -U transformers --progress-bar off
!pip install -qqq bitsandbytes accelerate

Cloning into 'mergekit'...
remote: Enumerating objects: 1287, done.
remote: Counting objects: 100% (715/715), done.
remote: Compressing objects: 100% (286/286), done.
remote: Total 1287 (delta 579), reused 475 (delta 428), pack-reused 572
Receiving objects: 100% (1287/1287), 365.57 KiB | 1.89 MiB/s, done.
Resolving deltas: 100% (870/870), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mergekit (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installe

In [2]:
merge_config  = """
base_model: unsloth/mistral-7b-instruct-v0.2-bnb-4bit
gate_mode: hidden
dtype: float16
experts:
  - source_model: akameswa/mistral-7b-instruct-javascript-4bit
    positive_prompts: ["You are helpful a coding assistant good at javascript"]
  - source_model: akameswa/mistral-7b-instruct-java-4bit
    positive_prompts: ["You are helpful a coding assistant good at java"]
  - source_model: akameswa/mistral-7b-instruct-cpp-4bit
    positive_prompts: ["You are helpful a coding assistant good at cpp"]
  - source_model: akameswa/mistral-7b-instruct-python-4bit
    positive_prompts: ["You are helpful a coding assistant good at python"]
"""

with open('config.yaml', 'w') as f:
    f.write(merge_config)

In [ ]:
!mergekit-moe config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

In [5]:
!pip install -qU huggingface_hub

from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template
import yaml

username = "akameswa"
MODEL_NAME = "mixtral-4x7b-instruct-code"
template_text = """
---
license: apache-2.0
tags:
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a MoE of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

```yaml
{{- merge_config -}}
```
"""

jinja_template = Template(template_text.strip())

data = yaml.safe_load(merge_config)
if "experts" in data:
    models = [expert["source_model"] for expert in data["experts"]]
else:
    raise Exception("No experts found in yaml config")

content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=merge_config,
    username=username,
)

card = ModelCard(content)
card.save('merge/README.md')

In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi

username = "akameswa"

api = HfApi(token="")

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)